In [ ]:
from typing import List, Tuple, Optional
from queue import PriorityQueue, deque

class MazeSolver:
    def __init__(self, maze: List[List[str]]):
        self.maze = maze
        self.rows = len(maze)
        self.cols = len(maze[0]) if maze else 0
        self.start = self.find_point('S')
        self.end = self.find_point('E')

    def find_point(self, symbol: str) -> Optional[Tuple[int,int]]:
        for r in range(self.rows):
            for c in range(self.cols):
                if self.maze[r][c] == symbol:
                    return (r, c)
        return None

    def is_valid(self, x: int, y: int) -> bool:
        return 0 <= x < self.rows and 0 <= y < self.cols and self.maze[x][y] != '#'

    def reconstruct_path(self, parent: dict, end: Tuple[int,int]) -> List[Tuple[int,int]]:
        path = []
        current = end
        while current is not None:
            path.append(current)
            current = parent.get(current)
        path.reverse()
        return path

    def bfs(self) -> Optional[List[Tuple[int,int]]]:
        if not self.start or not self.end:
            return None

        queue = deque([self.start])
        visited = set([self.start])
        parent = {self.start: None}

        while queue:
            x, y = queue.popleft()
            if (x, y) == self.end:
                return self.reconstruct_path(parent, (x, y))

            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                nx, ny = x + dx, y + dy
                if self.is_valid(nx, ny) and (nx, ny) not in visited:
                    queue.append((nx, ny))
                    visited.add((nx, ny))
                    parent[(nx, ny)] = (x, y)
        return None

    def dfs(self) -> Optional[List[Tuple[int,int]]]:
        if not self.start or not self.end:
            return None

        stack = [self.start]
        visited = set([self.start])
        parent = {self.start: None}

        while stack:
            x, y = stack.pop()
            if (x, y) == self.end:
                return self.reconstruct_path(parent, (x, y))

            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                nx, ny = x + dx, y + dy
                if self.is_valid(nx, ny) and (nx, ny) not in visited:
                    stack.append((nx, ny))
                    visited.add((nx, ny))
                    parent[(nx, ny)] = (x, y)
        return None

    def heuristic(self, a: Tuple[int,int], b: Tuple[int,int]) -> int:
        # Manhattan distance
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def a_star(self) -> Optional[List[Tuple[int,int]]]:
        if not self.start or not self.end:
            return None

        open_set = PriorityQueue()
        open_set.put((0, self.start))
        parent = {self.start: None}
        g_score = {self.start: 0}

        while not open_set.empty():
            _, current = open_set.get()

            if current == self.end:
                return self.reconstruct_path(parent, current)

            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                neighbor = (current[0] + dx, current[1] + dy)
                if self.is_valid(neighbor[0], neighbor[1]):
                    tentative_g_score = g_score[current] + 1
                    if tentative_g_score < g_score.get(neighbor, float('inf')):
                        parent[neighbor] = current
                        g_score[neighbor] = tentative_g_score
                        f_score = tentative_g_score + self.heuristic(neighbor, self.end)
                        open_set.put((f_score, neighbor))
        return None

    def print_maze_with_path(self, path: Optional[List[Tuple[int,int]]]) -> None:
        maze_copy = [row[:] for row in self.maze]
        if path:
            for x, y in path:
                if maze_copy[x][y] not in ('S', 'E'):
                    maze_copy[x][y] = '*'
        print('\n'.join(''.join(row) for row in maze_copy))

if __name__ == "__main__":
    maze = [
        ['S', '.', '.', '#', '.', '.', '.'],
        ['#', '#', '.', '#', '.', '#', '.'],
        ['.', '.', '.', '.', '.', '#', '.'],
        ['.', '#', '#', '#', '.', '#', '.'],
        ['.', '.', '.', '#', '.', '.', 'E']
    ]

    solver = MazeSolver(maze)

    for method in ['bfs', 'dfs', 'a_star']:
        print(f"\n--- Solving with {method.upper()} ---")
        if method == 'bfs':
            path = solver.bfs()
        elif method == 'dfs':
            path = solver.dfs()
        else:
            path = solver.a_star()

        if path:
            print(f"Path found! Length: {len(path)}")
            solver.print_maze_with_path(path)
        else:
            print("No path found.")
